In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import joblib
import pandas as pd
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

from hyperopt.pyll.base import scope
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from itertools import combinations
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import Pool, CatBoostClassifier
from kaggler.model import AutoLGB
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss
import random

# 1. 문제 정의

# 2. 데이터 수집

## (1) 데이콘 기본 데이터

In [ ]:
train = pd.read_csv('data/train.csv').drop(['index'], axis=1).fillna('NAN')
test = pd.read_csv('data/test.csv').drop(['index'], axis=1).fillna('NAN')
sample_submission = pd.read_csv('data/sample_submission.csv')

# 3. 탐색적 데이터 분석

# 4. 변수 조정

In [ ]:
# train데이터와 test데이터 변수를 함께 조정하기 위해 병합
merge_data = pd.concat([train, test], axis = 0)

# DAYS_BIRTH
merge_data['DAYS_BIRTH_month']=np.floor((-merge_data['DAYS_BIRTH'])/30)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/30)/12).astype(int)*12)
merge_data['DAYS_BIRTH_week']=np.floor((-merge_data['DAYS_BIRTH'])/7)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/7)/4).astype(int)*4)

# DAYS_EMPLOYED
merge_data['DAYS_EMPLOYED_month']=np.floor((-merge_data['DAYS_EMPLOYED'])/30)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['DAYS_EMPLOYED_week']=np.floor((-merge_data['DAYS_EMPLOYED'])/7)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

# before_EMPLOYED
merge_data['before_EMPLOYED']=merge_data['DAYS_BIRTH']-merge_data['DAYS_EMPLOYED']
merge_data['before_EMPLOYED_month']=np.floor((-merge_data['before_EMPLOYED'])/30)-(
    (np.floor((-merge_data['before_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['before_EMPLOYED_week']=np.floor((-merge_data['before_EMPLOYED'])/7)-(
    (np.floor((-merge_data['before_EMPLOYED'])/7)/4).astype(int)*4)

# DAYS_BIRTH
merge_data['DAYS_BIRTH_month/income_total'] = merge_data['DAYS_BIRTH_month'] / merge_data['income_total']
merge_data['DAYS_BIRTH_week/income_total'] = merge_data['DAYS_BIRTH_week'] / merge_data['income_total']

# DAYS_EMPLOYED
merge_data['DAYS_EMPLOYED_month/income_total'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED_week/income_total'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['income_total']

# before_EMPLOYED
merge_data['before_EMPLOYED/income_total'] = merge_data['before_EMPLOYED'] / merge_data['income_total']
merge_data['before_EMPLOYED_month/income_total'] = merge_data['before_EMPLOYED_month'] / merge_data['income_total']
merge_data['before_EMPLOYED_week/income_total'] = merge_data['before_EMPLOYED_week'] / merge_data['income_total']

merge_data['child_num/income_total'] = merge_data['child_num'] / merge_data['income_total']
merge_data['family_size/income_total'] = merge_data['family_size'] / merge_data['income_total']
merge_data['DAYS_BIRTH/income_total'] = merge_data['DAYS_BIRTH'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED/income_total'] = merge_data['DAYS_EMPLOYED'] / merge_data['income_total']
merge_data['begin_month/income_total'] = merge_data['begin_month'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED/DAYS_BIRTH'] =  merge_data['DAYS_EMPLOYED'] / merge_data['DAYS_BIRTH']

# 소득 skewed-data 처리
merge_data['log1p_income_total'] = np.log1p(merge_data['income_total'])
merge_data['log_income_total'] = np.log(merge_data['income_total'])
merge_data['sqrt_income_total'] = np.sqrt(merge_data['income_total'])
merge_data['boxcox_income_total'] = stats.boxcox(merge_data['income_total'])[0]

merge_data = merge_data.fillna(-999)
train = merge_data[merge_data['credit'] != -999]
test = merge_data[merge_data['credit'] == -999]
test.drop('credit', axis = 1, inplace = True)

train_cols = list(train.columns); train_cols.remove('credit'); train_cols.append('credit')
train = train[train_cols]

train = train[train['child_num']<=6].reset_index(drop=True) # 아이의 수가 7명 이상인 데이터 제거

## 인코딩

In [ ]:
train_oh = train.copy()
train_noh = train.copy()
test_oh = test.copy()
test_noh = test.copy()

In [ ]:
object_col = []
for col in train_noh.columns:
    if train_noh[col].dtype == 'object':
        train_noh[col] = train_noh[col].astype('category')
        test_noh[col] = test_noh[col].astype('category')

In [ ]:
object_col = []
for col in train_oh.columns:
    if train_oh[col].dtype == 'object':
        object_col.append(col)
print(object_col)        
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train_oh.drop(object_col, axis=1, inplace=True)
train_oh = pd.concat([train_oh, train_onehot_df], axis=1)    

test_onehot_df = pd.DataFrame(enc.transform(test_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test_oh.drop(object_col, axis=1, inplace=True)
test_oh = pd.concat([test_oh, test_onehot_df], axis=1)

## Feature 하나씩 빼면서 성능 체크

변수 하나씩 제거하면서 성능 체크 (제거하고 또 제거하고 해서 최적 변수들로 모델 돌리기)

In [ ]:
train_x = train_noh.drop(['credit'], axis=1)
train_y = train_noh['credit']
test_x = test_noh.copy()

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
cv = np.zeros((train_x.shape[0], 3))
for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
    x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
    y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
    lgbm = LGBMClassifier(n_estimators=1000, objective='multiclass')
    lgbm.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None)
    cv[val_idx] = lgbm.predict_proba(x_val)
print(f'Initial_multi_logloss: {log_loss(train_y, cv)}')
remove_features = []
for i in range(1, 2):
    for j in combinations(list(range(0, train_x.shape[1])), i):
        train_new_x = train_x.drop(train_x.columns[list(j)], axis=1)
        
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        cv = np.zeros((train_new_x.shape[0], 3))
        for n, (train_idx, val_idx) in enumerate(kfold.split(train_new_x, train_y)):
            x_train, x_val = train_new_x.iloc[train_idx], train_new_x.iloc[val_idx]
            y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
            lgbm = LGBMClassifier(n_estimators=1000, objective='multiclass')
            lgbm.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None)
            cv[val_idx] = lgbm.predict_proba(x_val)
        print(f'{j} multi_logloss: {log_loss(train_y, cv)}')

## 변수 지우고 다시 인코딩

In [ ]:
remove_features = [] # 지울 변수들 입력
# remove_features = [] # 지울 변수들 입력
train = train.drop(train.columns[remove_features], axis=1)
test = test.drop(test.columns[remove_features], axis=1)

In [ ]:
train_oh = train.copy()
train_noh = train.copy()
test_oh = test.copy()
test_noh = test.copy()

In [ ]:
object_col = []
for col in train_noh.columns:
    if train_noh[col].dtype == 'object':
        train_noh[col] = train_noh[col].astype('category')
        test_noh[col] = test_noh[col].astype('category')

In [ ]:
object_col = []
for col in train_oh.columns:
    if train_oh[col].dtype == 'object':
        object_col.append(col)
print(object_col)        
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train_oh.drop(object_col, axis=1, inplace=True)
train_oh = pd.concat([train_oh, train_onehot_df], axis=1)    

test_onehot_df = pd.DataFrame(enc.transform(test_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test_oh.drop(object_col, axis=1, inplace=True)
test_oh = pd.concat([test_oh, test_onehot_df], axis=1)

# 6. 모델 학습

In [ ]:
pred_dict = {}
pred_test_dict = {}

## (1) Lightgbm

In [ ]:
train_x = train_noh.drop(['credit'], axis=1)
train_y = train_noh['credit']
test_x = test_noh.copy()

### 튜닝은 hyperopt 이거로 하시면 됩니다!

In [ ]:
SEED=np.random.randint(1, 100)

# Hyperopt의 metric함수를 StratifiedKFold(cv=5)로 구하기
def score(params):
    print("Training with params: ")
    print(params)
    
    kfold = StratifiedKFold(n_splits=5, random_state = SEED, shuffle = True)
    cv = np.zeros((train_x.shape[0], 3))

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

        lgbmodel = LGBMClassifier(**params)

        lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=100) 
        cv[val_idx, :] = lgbmodel.predict_proba(x_val)
        print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
    print('multi_logloss:', log_loss(train_y, cv))
    score = log_loss(train_y, cv)
    return {'loss': score, 'status': STATUS_OK}

# Hyperopt의 범위를 지정해주고 max_evals만큼 반복한 후 최적의 파라미터를 반환
def optimize(random_state=SEED):
    
    space = {
        'learning_rate': hp.quniform('learning_rate', 0.003, 0.006, 0.001),
        'num_leaves': scope.int(hp.quniform('num_leaves', 1000, 1200, 50)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
        'subsample': hp.quniform('subsample', 0.8, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.3, 0.7, 0.05),
        'max_depth' : -1,
        'n_estimators' : 5000,
        'objective' : 'multiclass',
        'num_class' : 3,
        'seed': SEED,
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                # trials=trials, 
                max_evals=12)
    return best

lgb_best_hyperparams = optimize()
print("The best hyperparameters are: ", "\n")
print(lgb_best_hyperparams)

In [ ]:
lucky_seeds=[0, 100, 1000]
for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=10, random_state = seed, shuffle = True)
    cv=np.zeros((train_x.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):

        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        lgbmodel = LGBMClassifier(**lgb_best_hyperparams)

        lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=100) 

        # CROSS-VALIDATION , EVALUATE CV
        cv[val_idx,:] = lgbmodel.predict_proba(x_val)
        pred_test += lgbmodel.predict_proba(test_x) / 10
    pred_dict['lgb'+str(i+1)] = cv
    pred_test_dict['lgb'+str(i+1)] = pred_test
        
    print('multi_logloss :', log_loss(train_y, cv))

## (2) XGBoost

원핫인코딩된 feature로 만들어주기 **꼭 밑에 코드 실행하고 XGBoost랑 Randomforest 돌리기!!**

In [ ]:
train_x = train_oh.drop(['credit'], axis=1)
train_y = train_oh['credit']
test_x = test_oh.copy()

In [ ]:
SEED=np.random.randint(1, 100)

# Hyperopt의 metric함수를 StratifiedKFold(cv=5)로 구하기
def score(params):
    print("Training with params: ")
    print(params)
    
    kfold = StratifiedKFold(n_splits=5, random_state = SEED, shuffle = True)
    cv = np.zeros((train_x.shape[0], 3))

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        dtrain = xgb.DMatrix(x_train, label=y_train)
        dvalid = xgb.DMatrix(x_val, label=y_val)
        watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

        xgbmodel = xgb.train(params, dtrain, 100000, watchlist, early_stopping_rounds=30, verbose_eval=None)
        cv[val_idx, :] = xgbmodel.predict(dvalid)
        print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
    print('multi_logloss:', log_loss(train_y, cv))
    score = log_loss(train_y, cv)
    return {'loss': score, 'status': STATUS_OK}

# Hyperopt의 범위를 지정해주고 max_evals만큼 반복한 후 최적의 파라미터를 반환
def optimize(random_state=SEED):
    
#     param = {'objective':'multi:softprob', 'seed':SEED, 'num_class': 3, 'eval_metric':'mlogloss', 
#          'eta': 0.01, 'min_child_weight': 3,
#          'colsample_bytree': 0.3, 'colsample_bylevel': 0.6, 'subsample': 0.8
#         }
    space = {
        'eta': hp.quniform('eta', 0.003, 0.006, 0.001),
        #'max_depth':  hp.choice('max_depth', np.arange(20, 30, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 3, 1),
        'subsample': hp.quniform('subsample', 0.6, 0.8, 0.05),
        'gamma': hp.quniform('gamma', 0.6, 0.8, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.3, 0.7, 0.05),
        'colsample_bylevel': hp.quniform('colsample_bylevel', 0.3, 0.7, 0.05),
        'max_depth' : 100,
        'eval_metric': 'mlogloss',
        'objective' : 'multi:softprob',
        'num_class' : 3,
        'seed': SEED,
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                # trials=trials, 
                max_evals=10)
    return best

xgb_best_hyperparams = optimize()
print("The best hyperparameters are: ", "\n")
print(xgb_best_hyperparams)

### 3 seeds x 5 folds

In [ ]:
lucky_seeds=[0, 100, 1000]
xgtest = xgb.DMatrix(test_x)
for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=10, random_state = seed, shuffle = True)
    cv = np.zeros((train.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        dtrain = xgb.DMatrix(x_train, label=y_train)
        dvalid = xgb.DMatrix(x_val, label=y_val)
        watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
        
        xgbmodel = xgb.train(xgb_best_hyperparams, dtrain, 100000, watchlist, 
                             early_stopping_rounds=30, verbose_eval=None)

        cv[val_idx, :] = xgbmodel.predict(dvalid)
        print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
        pred_test += xgbmodel.predict(xgtest) / 10
        
    pred_dict['xgb'+str(i+1)] = cv
    pred_test_dict['xgb'+str(i+1)] = pred_test
    print('multi_logloss:', log_loss(train_y, cv))

## (3) Random Forest

### 3 seeds, 5 folds

In [ ]:
lucky_seeds=[0, 100, 1000]
for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=10, random_state = seed, shuffle = True)
    cv = np.zeros((train_x.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

        rfmodel = RandomForestClassifier(n_estimators=1200, criterion='entropy', max_depth=60, random_state=seed)
        rfmodel.fit(x_train, y_train)
        
        cv[val_idx, :] = rfmodel.predict_proba(x_val)      
        print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
        pred_test += rfmodel.predict_proba(test_x) / 10
        
    pred_dict['rf'+str(i+1)] = cv
    pred_test_dict['rf'+str(i+1)] = pred_test
    print('multi_logloss :', log_loss(train_y, cv))

## (4) Stacking (AutoLGB)

In [ ]:
X_train = pd.DataFrame(np.hstack([x for _, x in pred_dict.items()]))
X_test = pd.DataFrame(np.hstack([x for _, x in pred_test_dict.items()]))

pred = np.zeros((X_train.shape[0], 3), dtype=float)
pred_test = np.zeros((X_test.shape[0], 3), dtype=float)
#kfold = KFold(n_splits=5, random_state = seed, shuffle = True)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i_cv, (i_trn, i_val) in enumerate(cv.split(X_train, train_y)):
    if i_cv == 0:
        clf = AutoLGB(objective='multiclass', metric='multi_logloss', params={'num_class': 3}, 
                      feature_selection=False, n_est=10000)
        clf.tune(X_train.iloc[i_trn], train_y[i_trn])
        n_best = clf.n_best
        features = clf.features
        params = clf.params
        print(f'best iteration: {n_best}')
        print(f'selected features ({len(features)}): {features}')        
        print(params)
        clf.fit(X_train.iloc[i_trn], train_y[i_trn])
    else:
        train_data = lgb.Dataset(X_train[features].iloc[i_trn], label=train_y[i_trn])
        clf = lgb.train(params, train_data, n_best, verbose_eval=100)
    
    pred[i_val] = clf.predict(X_train[features].iloc[i_val])
    pred_test += clf.predict(X_test[features]) / 5

In [ ]:
print(f'CV Log Loss: {log_loss(train_y, pred):.6f}')

# 결과 제출

In [ ]:
submission = sample_submission.copy()
submission.iloc[:, 1:] = pred_test

In [ ]:
submission.to_csv('submission/hongbi.csv', index=False)